# Exploratory Data Analysis for METR-IMC

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_hdf("./datasets/metr_imc/metr-imc.h5")
data.head().reset_index().iloc[:, :9]


,index,1640040200,1670005201,1670015500,1640007700,1660028600,1650013200,1640038100,1630011802
0,2023-12-25 00:00:00,0.0,318.0,17.0,NaN,0.0,23.0,6.0,0.0
1,2023-12-25 01:00:00,0.0,284.0,13.0,NaN,0.0,15.0,6.0,4.0
2,2023-12-25 02:00:00,0.0,202.0,7.0,NaN,0.0,10.0,0.0,16.0
3,2023-12-25 03:00:00,0.0,150.0,4.0,NaN,0.0,14.0,0.0,0.0
4,2023-12-25 04:00:00,0.0,129.0,7.0,NaN,0.0,0.0,0.0,0.0


# 